In [16]:
import requests
import os
import json
from minio import Minio
import io
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("TWITTER_BEARER_TOKEN")

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    ETH_search_rules = [
        {
            "value": "#ETH OR #ethereum OR #eth"
        },
    ]
    payload = {"add": ETH_search_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    i = 0
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            formatted = json.dumps(json_response, indent=4, sort_keys=True)
            print(formatted)
            s3_client.put_object(bucket, f"test_tweets/{i}.json", io.BytesIO(bytes(formatted, "utf-8")), len(formatted))
            i = i + 1

In [17]:
s3_access_key = os.environ.get("S3_ACCESS_KEY")
s3_secret_key = os.environ.get("S3_SECRET_KEY")
bucket = "twitter-data-123456"
s3_client = Minio("storage.googleapis.com", access_key=s3_access_key, secret_key=s3_secret_key)

In [18]:
rules = get_rules()
delete = delete_all_rules(rules)
set = set_rules(delete)

{"data": [{"id": "1486510236231888900", "value": "#ETH OR #ethereum OR #eth"}], "meta": {"sent": "2022-01-27T01:24:02.815Z", "result_count": 1}}
{"meta": {"sent": "2022-01-27T01:24:03.176Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "#ETH OR #ethereum OR #eth", "id": "1486510280108503041"}], "meta": {"sent": "2022-01-27T01:24:03.477Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}


In [19]:
get_stream(set)

<Response [429]>
429


Exception: Cannot get stream (HTTP 429): {"title":"ConnectionException","detail":"This stream is currently at the maximum allowed connection limit.","connection_issue":"TooManyConnections","type":"https://api.twitter.com/2/problems/streaming-connection"}